# Random Forest
(C) 2018 Dariusz Kajtoch

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sys
sys.path.append('../')
from incentives import optimal_revenue

In [2]:
data = pd.read_csv('../data/train.csv')

In [3]:
X = data[[
          'perc_premium_paid_by_cash_credit', 
          'age_in_days', 
          'Income', 
          'Count_3-6_months_late', 
          'Count_6-12_months_late', 
          'Count_more_than_12_months_late', 
          'no_of_premiums_paid', 
#          'premium',
          'application_underwriting_score',
#          'sourcing_channel'
         ]]
#X['logIncome'] = np.log(data['Income'])
#X['logAge'] = np.log(data['age_in_days'])
#X['logperc_premium_paid_by_cash_credit'] = np.log(1. + data['perc_premium_paid_by_cash_credit'])
#X['standard_age'] = data['age_in_days'].apply(lambda row: np.floor(row/365.))
#X['perc'] = data['perc_premium_paid_by_cash_credit'].apply(lambda x: x if x<0.5 else x-1.)

#from scipy.stats import boxcox

#X['BCage_in_days'], _ = boxcox(data['age_in_days'])
#X['BCIncome'], _ = boxcox(data['Income'])

# fill nans
for name in X.columns.tolist():
    val = X[name].mean()#.iloc[0]
    X[name].fillna(val, inplace=True)
    
# predictor
y = data['renewal']

In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss

# criterion = 'gini', 'entropy'
# max_dept
# min_samples_split=2
# min_samples_leaf=1
# max_features

estimator = RandomForestClassifier(n_estimators=250,
                                   criterion='gini',
#                                   max_features='auto',
                                   min_samples_leaf=90,
                                   min_samples_split=3,
                                   random_state=1563
#                                   max_depth=3
                                  )
print(estimator.get_params)

# ----------------------
# Validate
# ----------------------

auc_tab  = np.array([])
loss_tab = np.array([])
revenue  = np.array([])

random_state = [245, 34, 78, 1234, 888]

for rstate in random_state:
    skf = StratifiedKFold(n_splits=10, random_state=rstate)

    for train_index, test_index in skf.split(X,y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        estimator.fit(X_train, y_train)
        prob = estimator.predict_proba(X_test)
    
        auc = roc_auc_score(y_test, prob[:,1])
        loss = log_loss(y_test, prob)
        net = np.array([optimal_revenue(premium_test.iloc[i], prob[i,1])[1] for i in range(0,len(prob))])
        net = np.sum(net)
    
        auc_tab  = np.append(auc_tab, auc)
        loss_tab = np.append(loss_tab, loss)
        revenue  = np.append(revenue, net)
    
        print('Single Fold done')

<bound method RandomForestClassifier.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=90, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=1563, verbose=0,
            warm_start=False)>
Single Fold done
Single Fold done
Single Fold done
Single Fold done
Single Fold done
Single Fold done
Single Fold done
Single Fold done
Single Fold done
Single Fold done


In [12]:
print('AUC: %.8f +/- %.8f' % (np.mean(auc_tab), np.std(auc_tab)))
print('Loss: %.8f +/- %.8f' % (np.mean(loss_tab), np.std(loss_tab)))
print('-------------------------------------------')
print(estimator.feature_importances_)

AUC: 0.84255586 +/- 0.00838847
Loss: 0.17947660 +/- 0.00279630
-------------------------------------------
[0.19457891 0.03810079 0.03262838 0.15309797 0.3372076  0.16029443
 0.03855606 0.04553585]


1. n_estimators = 250, min_samples_leaf = 100, min_samples_split = 3
    * AUC: 0.84258960 +/- 0.00814210
    * Loss: 0.17958005 +/- 0.00274064
2. n_estimators = 250, min_samples_leaf = 120, min_samples_split = 3
    * AUC: 0.84265138 +/- 0.00821999
    * Loss: 0.17969945 +/- 0.00269591
3. n_estimators = 270, min_samples_leaf = 100, min_samples_split = 3
    * AUC: 0.84255165 +/- 0.00816924
    * Loss: 0.17959528 +/- 0.00273818
1. n_estimators = 250, min_samples_leaf = 90, min_samples_split = 3
    * AUC: 0.84255586 +/- 0.00838847
    * Loss: 0.17947660 +/- 0.00279630